# K-Means Algorithm Implementation

A simple [K-Means clustering](https://en.wikipedia.org/wiki/K-means_clustering) implementation and performance comparison with [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [ ]:
import matplotlib.pyplot as plot
from numpy import divide, int8, newaxis, ones, sum, zeros
from numpy.random import rand, randint
from sklearn.cluster import KMeans

from utility import euclidean

## Self-implemented K-Means clustering method

In [ ]:
def cluster(data, cluster_count, max_iteration=1000):
    data_count = data.shape[0]
    # select the clusters randomly from data
    clusters = data[randint(data_count, size=cluster_count), :]
    # assign random memberships to the data
    memberships = zeros(data_count, dtype=int8)
    iteration = 0
    _inertia = 1e308
    # k-means loop starting
    while True:
        changed = False
        # reset new cluster variables
        _clusters = zeros((cluster_count, data.shape[1]))
        _cluster_size = zeros(cluster_count)

        # CLUSTER ASSIGNMENT STEP
        # assign each data to the nearest cluster
        for i, datum in enumerate(data):
            dmin = float("Inf")
            # find the smallest distance cluster center
            for j, cluster in enumerate(clusters):
                distance = euclidean(datum, cluster)
                if distance < dmin:
                    dmin = distance
                    n = j
            # assign closest cluster to the datum
            if memberships[i] != n:
                memberships[i] = n
                changed = True
            # store the sum of the all data belonging to the same cluster
            _clusters[memberships[i]] = _clusters[memberships[i]] + datum
            # store the data count of cluster
            _cluster_size[memberships[i]] += 1

        # UPDATE STEP
        # calculate new cluster centers using data cluster information
        clusters = divide(_clusters, _cluster_size[:, newaxis])

        # COST CALCULATION
        inertia = sum((data - clusters[memberships]) ** 2)
        print(f"iteration: {iteration} cost: {inertia}")
        if _inertia == inertia:
            break
        else:
            _inertia = inertia
        iteration += 1
        # check for stop criteria
        if iteration > max_iteration or changed is False:
            break
    # print final inertia
    inertia = sum((data - clusters[memberships]) ** 2)
    # data cluster memberships and cluster centers are returned
    return clusters, memberships, inertia

## Testing

### Generate random data

In [ ]:
data = rand(25, 2)

### Run self-implementation

In [ ]:
clusters, memberships, inertia = cluster(data, 2)
print(f"Self-implementation inertia:{inertia}")

### Run scikit-learn implementation

In [ ]:
sklKMeans = KMeans(n_clusters=2)
sklKMeans.fit(data)
print(f"scikit-learn inertia: {sklKMeans.inertia_}")

### Compare results by plotting

In [ ]:
f, (ax1, ax2) = plot.subplots(1, 2, sharey=True)
ax1.scatter(data[:, 0], data[:, 1], c=memberships)
ax1.plot(clusters[:, 0], clusters[:, 1], "g^")
ax1.set_title("self-implementation")
ax2.scatter(data[:, 0], data[:, 1], c=sklKMeans.labels_)
ax2.plot(sklKMeans.cluster_centers_[:, 0], sklKMeans.cluster_centers_[:, 1], "g^")
ax2.set_title("scikit-learn")
plot.show()